In [9]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import (Dense,Flatten,Activation,InputLayer,Masking)

In [10]:
data, info = tfds.load('smartwatch_gestures', split='train', with_info=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3251 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/smartwatch_gestures/1.0.0.incomplete1V6RKL/smartwatch_gestures-train.tfrec…

Dataset smartwatch_gestures downloaded and prepared to /root/tensorflow_datasets/smartwatch_gestures/1.0.0. Subsequent calls will reuse this data.


In [11]:
info

tfds.core.DatasetInfo(
    name='smartwatch_gestures',
    full_name='smartwatch_gestures/1.0.0',
    description="""
    The **SmartWatch Gestures Dataset** has been collected to evaluate several
    gesture recognition algorithms for interacting with mobile applications using
    arm gestures.
    
    Eight different users performed twenty repetitions of twenty different gestures,
    for a total of 3200 sequences. Each sequence contains acceleration data from the
    3-axis accelerometer of a first generation Sony SmartWatch™, as well as
    timestamps from the different clock sources available on an Android device. The
    smartwatch was worn on the user's right wrist. The gestures have been manually
    segmented by the users performing them by tapping the smartwatch screen at the
    beginning and at the end of every repetition.
    """,
    homepage='https://tev.fbk.eu/resources/smartwatch',
    data_dir=PosixGPath('/tmp/tmptnj86fxvtfds'),
    file_format=tfrecord,
    download

In [85]:
def preprocess(example):
  out = []
  desired_shape = 55
  feats = example['features']
  label = example['gesture']
  label = tf.expand_dims(label, axis=0)
  #print("label_shape=",label.shape, label)
  ks = list(feats.keys())
  for j in range(len(ks)):
    if(j<3):
      normalized = feats[ks[j]]
    else:
      normalized = feats[ks[j]]/tf.reduce_max(feats[ks[j]])

    padding_size = desired_shape - len(normalized)
    #print(padding_size)
    padding_tensor = tf.zeros(padding_size, dtype=normalized.dtype)
    padded_tensor = tf.concat([normalized, padding_tensor], axis=0)
    out.append(padded_tensor)
  out = tf.convert_to_tensor(out)
  reshaped = tf.reshape(out,(1,desired_shape*6))
  return reshaped,label

In [91]:
train_dataset = data.map(preprocess).batch(32)

In [93]:
def create_model():
  model = tf.keras.Sequential([
      Dense(64,activation='relu',input_shape=(1,330)),
      Dense(32,activation='relu'),
      Dense(20,activation='softmax')
  ])
  return model

In [94]:
model = create_model()
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [95]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1, 64)             21184     
                                                                 
 dense_1 (Dense)             (None, 1, 32)             2080      
                                                                 
 dense_2 (Dense)             (None, 1, 20)             660       
                                                                 
Total params: 23924 (93.45 KB)
Trainable params: 23924 (93.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [96]:
model.fit(train_dataset,epochs=5,batch_size=32)

Epoch 1/5
102/102 [==============================] - 2s 8ms/step - loss: 2.3385 - accuracy: 0.3270
Epoch 2/5
102/102 [==============================] - 1s 9ms/step - loss: 0.8110 - accuracy: 0.7588
Epoch 3/5
102/102 [==============================] - 1s 9ms/step - loss: 0.4250 - accuracy: 0.8840
Epoch 4/5
102/102 [==============================] - 1s 8ms/step - loss: 0.2795 - accuracy: 0.9274
Epoch 5/5
102/102 [==============================] - 1s 5ms/step - loss: 0.1979 - accuracy: 0.9536
